# CIFAR10 CNN

# RMSProp, Dataset Augmentation, ReduceLROnPlateau

In [9]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import os
from time import time

# Training Params
batch_size = 32
num_classes = 10
epochs = 200
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_cnn_trained_model.h5'
initial_lr=0.0004

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = keras.optimizers.rmsprop(lr=initial_lr, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Use dataset augmentation
datagen = ImageDataGenerator(
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Tensorboard Callback
tbCallBack = keras.callbacks.TensorBoard(
    batch_size=batch_size,
    log_dir="logs/{}".format(time()),
    histogram_freq=0,
    write_graph=True,
    write_images=True,
    update_freq='batch'
)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [tbCallBack, lr_reducer]

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4,
                    callbacks=callbacks)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Epoch 1/200
1563/1562 [==============================] - 18s 11ms/step - loss: 1.4359 - acc: 0.5206 - val_loss: 1.1409 - val_acc: 0.6144
Epoch 2/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.9614 - acc: 0.6686 - val_loss: 1.1649 - val_acc: 0.6063
Epoch 3/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.8323 - acc: 0.7161 - val_loss: 0.8209 - val_acc: 0.7254
Epoch 4/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.7487 - acc: 0.7449 - val_loss: 0.7465 - val_acc: 0.7482
Epoch 5/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.6876 - acc: 0.7670 - val_loss: 0.7725 - val_acc: 0.7441
Epoch 6/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.6401 - acc: 0.7824 - val_loss: 0.6752 - val_acc: 0.7693
Epoch 7/200
1563/1562 [==============================] - 18s 11ms/step - loss: 0.6037 - acc: 0.7955 - val

Epoch 60/200
1563/1562 [==============================] - 18s 11ms/step - loss: 0.1778 - acc: 0.9391 - val_loss: 0.4730 - val_acc: 0.8560
Epoch 61/200
1563/1562 [==============================] - 18s 11ms/step - loss: 0.1757 - acc: 0.9401 - val_loss: 0.4728 - val_acc: 0.8558
Epoch 62/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.1743 - acc: 0.9398 - val_loss: 0.4737 - val_acc: 0.8559
Epoch 63/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.1725 - acc: 0.9409 - val_loss: 0.4731 - val_acc: 0.8558
Epoch 64/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.1743 - acc: 0.9392 - val_loss: 0.4713 - val_acc: 0.8553
Epoch 65/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.1736 - acc: 0.9405 - val_loss: 0.4752 - val_acc: 0.8555
Epoch 66/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.1739 - acc: 0.9405 - val_loss: 0.4728 - val_acc: 0.8559
Epoch 67/200
1563/1562 [==========

1563/1562 [==============================] - 17s 11ms/step - loss: 0.1728 - acc: 0.9409 - val_loss: 0.4753 - val_acc: 0.8547
Epoch 179/200
1563/1562 [==============================] - 16s 10ms/step - loss: 0.1706 - acc: 0.9405 - val_loss: 0.4750 - val_acc: 0.8558
Epoch 180/200
1563/1562 [==============================] - 16s 11ms/step - loss: 0.1730 - acc: 0.9399 - val_loss: 0.4732 - val_acc: 0.8553
Epoch 181/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.1703 - acc: 0.9420 - val_loss: 0.4749 - val_acc: 0.8554
Epoch 182/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.1759 - acc: 0.9394 - val_loss: 0.4749 - val_acc: 0.8554
Epoch 183/200
1563/1562 [==============================] - 17s 11ms/step - loss: 0.1715 - acc: 0.9408 - val_loss: 0.4733 - val_acc: 0.8557
Epoch 184/200
1563/1562 [==============================] - 16s 11ms/step - loss: 0.1768 - acc: 0.9402 - val_loss: 0.4731 - val_acc: 0.8549
Epoch 185/200
1563/1562 [================